# Generative Question Answering






## Installation

1. Click File -> Save a Copy in Drive. This will create a copy for you to modify in your Google Drive.

2. This project will require you to install langchain, cohere, chromadb, and other libraries. Run the command below to start the installation. Installation may take a couple of minutes.


In [1]:
!pip install cohere nltk unstructured langchain chromadb langchain-community langchain-cohere


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import Libraries




In [2]:
import os
import nltk
nltk.download('punkt')

# from langchain import Cohere
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_cohere import CohereEmbeddings #NEW
from langchain_cohere import ChatCohere #New

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mdev\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Setup

Before running the next cell, make sure you have a [Cohere API key](https://dashboard.cohere.ai/api-keys). Enter your API key using getpass in the cell below.


In [3]:
from getpass import getpass
api_key = getpass()
os.environ["COHERE_API_KEY"] = api_key

## Uploading documents to Colab

There are two ways you can upload your documents to Colab.

#### Method 1

1. Right-click on the link below and save the text file to your computer.

[Wikipedia Article: Quantum Computing](https://raw.githubusercontent.com/Thinkful-Ed/ai-in-web-dev-resources/refs/heads/main/books/quantumcomputers.txt)

2. Click on the folder icon tab at the left of the Colab editor. Create a new *documents* directory. Click on the *upload file* icon and select the file that you want to upload.

#### Method 2

Run the command below:

In [4]:
!curl -o ./quantumcomputers.txt https://raw.githubusercontent.com/Thinkful-Ed/ai-in-web-dev-resources/refs/heads/main/books/quantumcomputers.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  115k  100  115k    0     0   127k      0 --:--:-- --:--:-- --:--:--  127k


## Load the documents

The [DirectoryLoader](https://js.langchain.com/docs/api/document_loaders_fs_directory/classes/DirectoryLoader) class will allow you to load multiple documents that are in a directory. Create a directory by clicking in the folder icon at the left and upload your text documents there. If you want to use another format, such as `md` files, make sure to change the `glob="**/*.txt"` variable and read the documentation to make sure that the file type is supported.

**Note:** Cohere is now imposing a limit on embeddings. If you get an error try a smaller text (like a wikipedia article).

In [5]:
loader = TextLoader("./quantumcomputers.txt", encoding="utf-8")
documents = loader.load()
print (f'You have {len(documents)} document(s) in your documents folder')

You have 1 document(s) in your documents folder


## Splitting Text

Splitting text is useful because when you have a long document, it can be difficult to find the specific information relevant to a question. The `CharacterTextSplitter` helps break down the text into smaller, more manageable chunks.

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(documents)
print (f'Your document(s) was/were splitted into {len(texts)} chunks.')

Your document(s) was/were splitted into 167 chunks.


## Embeddings and RetrievalQA

In simple terms, this code sets up the necessary tools for question-answering. It converts text into numerical representations called embeddings, creates a search index for the documents, and sets up a retrieval-based question-answering system using a specific type of chain (RetrievalQA).


In [7]:
embeddings = CohereEmbeddings(
    cohere_api_key="OrU9yPWradw31QYCeXtQavimy6jV2EbsXRmYSHvz", #TODO Remove API Key
    model="embed-english-v3.0"  # Required model name
)
docsearch = Chroma.from_documents(texts, embeddings, persist_directory = 'db')
docsearch.persist() # save db
llm = ChatCohere(cohere_api_key="OrU9yPWradw31QYCeXtQavimy6jV2EbsXRmYSHvz") #TODO Remove API Key

qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=docsearch.as_retriever(),
    return_source_documents=True
)


C:\Users\mdev\AppData\Local\Temp\ipykernel_12232\2079752960.py:6: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  docsearch.persist() # save db


In [8]:
query = "what is quantum computing?"
result = qa.invoke({"query": query})
result['result']

'Quantum computing is a type of computing that exploits quantum mechanical phenomena to perform calculations. Unlike classical computers, which use bits as the basic unit of information (represented as either 0 or 1), quantum computers use quantum bits, or **qubits**. Qubits can exist in multiple states simultaneously due to **superposition**, allowing quantum computers to process a vast number of possibilities at once. Additionally, qubits can be **entangled**, meaning the state of one qubit is directly related to the state of another, even if they are physically far apart. This property enables quantum computers to perform certain types of calculations much faster than classical computers.\n\nQuantum computing has the potential to solve complex problems that are currently intractable for classical computers, such as factoring large numbers (which underpins many encryption schemes), simulating quantum systems, optimizing large systems, and solving certain types of mathematical problem

You can also get the sources:

In [9]:
result['source_documents']

[Document(metadata={'source': './quantumcomputers.txt'}, page_content='A \'\'\'quantum computer\'\'\' is a [[computer]] that exploits [[quantum mechanical]] phenomena. On small scales, physical matter exhibits properties of [[wave-particle duality|both particles and waves]], and quantum computing leverages this behavior using specialized hardware. [[Classical physics]] cannot explain the operation of these quantum devices, and a scalable quantum computer could perform some calculations [[Exponential growth|exponentially]] faster{{efn|As used in this article, "exponentially faster" has a precise [[computational complexity|complexity theoretical]] meaning. Usually, it means that as a function of input size in bits, the best known classical algorithm for a problem requires an [[exponential growth|exponentially growing]] number of steps, while a quantum algorithm uses only a polynomial number of steps.}} than any modern "classical" computer. In particular, a large-scale quantum computer co

In [10]:
sources = ""
for count, source in enumerate(result['source_documents'],1):
  sources += "Source " + str(count) + "\n"
  sources += source.page_content + "\n"

print(sources)


Source 1
A '''quantum computer''' is a [[computer]] that exploits [[quantum mechanical]] phenomena. On small scales, physical matter exhibits properties of [[wave-particle duality|both particles and waves]], and quantum computing leverages this behavior using specialized hardware. [[Classical physics]] cannot explain the operation of these quantum devices, and a scalable quantum computer could perform some calculations [[Exponential growth|exponentially]] faster{{efn|As used in this article, "exponentially faster" has a precise [[computational complexity|complexity theoretical]] meaning. Usually, it means that as a function of input size in bits, the best known classical algorithm for a problem requires an [[exponential growth|exponentially growing]] number of steps, while a quantum algorithm uses only a polynomial number of steps.}} than any modern "classical" computer. In particular, a large-scale quantum computer could [[post-quantum cryptography|break widely used encryption schemes

## Download the database

Zip the database into `db.zip` and download it to your computer.


In [11]:
import shutil
shutil.make_archive('db', 'zip', root_dir='db')  # zips the folder named 'db' into db.zip

'c:\\Users\\mdev\\OneDrive - Chegg Inc\\Documents\\coding\\artificial_intelligence\\chegg_resources\\db.zip'

You should see a zipped folder appear inside your folder where you ran the above cell